In [1]:
#This is only relevant if tomsup was installed from the github folder
#Move up to the correct folder
import os
os.chdir('..')
#And import the tomsup library
import tomsup as ts

#Get the competitive penny game payoff matrix
penny = ts.PayoffMatrix("penny_competitive")

# On Initial Internal States and how to change Them
The simulated ToM agents in the tomsup package use agnostic beliefs about their opponent's level probabilities $\lambda$ and choice probabilities $\mu$, while parameter estimation means $\mu^{\theta}$ are set to 1. All variances $\Sigma^{\theta}$ and $\Sigma$ for parameter and choice probability estimation, respectively, are also set to 1. Gradients for all parameters are 0 on the first trial, which means that no parameter estimates and variances are updated during the first trial. The tomsup package uses the same parameter values to the ones used in the VBA toolbox (Daunizeau et al., 2014) to enable comparison, but other settings are possible.

While you can give initial states directly to `ts.TOM()` using the priors. It can be problematic to get the correct input format, we instead suggest you initialize the model with the defaults and then change the internal states to match your desired belief:

In [2]:
tom_1 = ts.TOM(level=1)
init_states = tom_1.get_internal_states()
init_states

{'opponent_states': {0: {'opponent_states': {},
   'own_states': {'p_op_mean0': 0,
    'p_op_var0': 0,
    'p_self': nan,
    'p_op': nan}}},
 'own_states': {'p_k': array([1.]),
  'p_op_mean': array([0]),
  'param_mean': array([[0., 0., 0.]]),
  'param_var': array([[0., 0., 0.]]),
  'gradient': array([[0., 0., 1.]]),
  'p_self': nan,
  'p_op': nan}}

For instance, a variable which one might wish to change is the estimation of it's opponent sophistication level when playing against humans. This is more likely to be 1 as opposed to 0. This it the variable `priors["own_states"]["p_k"]` which is set to `[0.5, 0.5]` by default, i.e. agnostic. A more reasonable estimate for humans would favor the sophistication level 1. We can change it as follows:


In [3]:
init_states["own_states"]["p_k"] = [0.3, 0.7]
tom_1.set_internal_states(init_states)

In [4]:
# print the changed states
tom_1.print_internal()

opponent_states
|   0-ToM
|   |   opponent_states
|   |   own_states
|   |   |   p_op_mean0 (log odds):          0
|   |   |   p_op_var0 (log scale):          0
|   |   |   p_self (probability):           nan
|   |   |   p_op (probability):             nan
own_states
|   p_k (probability):              [0.3, 0.7]
|   p_op_mean (log odds):           [0]
|   param_mean:                     [[0.0, 0.0, 0.0]]
|   param_var (log scale):          [[0.0, 0.0, 0.0]]
|   gradient:                       [[0.0, 0.0, 0.999999997998081]]
|   p_self (probability):           nan
|   p_op (probability):             nan


for an explanation of the internal states besides the  we recommend you examine 

In [5]:
help(tom_1.print_internal)

Help on method print_internal in module tomsup.agent:

print_internal(keys: Union[list, NoneType] = None, level: Union[list, NoneType] = None) method of tomsup.agent.TOM instance
    keys (list | None): is the keys which you desire to print.
    level (list | None):  list of integers containing levels to print
    None indicate all levels will be printed.
    
    Explanation of internal states:
    opponent_states: indicate that the following states belong to the
    simulated opponent
    own_states: indicate that the states belongs to the agent itself
    p_k: is the estimated sophistication level of the opponent.
    p_op_mean: The mean estimate of the opponents choice probability in
    log odds.
    param_mean: the mean estimate of opponent parameters (in the scale
    used for the given parameter). If estimating another k-ToM the order
    of estimates is 1) Volatility, 2) Behavioural temperature, 3) Dilution,
    4) Bias. Note that bias is 3) if Dilution is not estimated.
    p